# evtl. TO DO --> Pfade anpassen; Batchsize, BIldgröße, Epochanzahl bestimmen

In [ ]:
data_dir=r'/content/drive/MyDrive/ComputerVision/' # Hier sind die Daten gespeichert. Der Ordner "ComputerVision" sollte jeweilis zu jeder Klasse einen unterordner enthalten -->zB. ComptuerVision/Auto und CoputerVison/keinAuto
base_path_für_logs_und_callbacks = r"/content/drive/MyDrive/Callbacks_und_Logs" # in diesem Ordner werden die Trainingsergebnisse des tages gespeichert
base_path_für_eingestellte_parameter = r"/content/drive/MyDrive/Eingestellte_parameter" #hier werden die Parameter gespeichert, die wir dokumentieren sollen

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Folgende grosgeschreibenen Parameter sind zu Dokumentieren (von Libewirth aus)
--> werden automatisch in parameter.txt datei gespeichert (speicherort:base_path_für_eingestellte_parameter)
- Batchsize
- Epochen anzahl
- Input resulution (laut ChatGPT--> Die bildgröße, also der paraemter PIC_SIZE)

Hinweis: LearningRate wird beim Modeltraining bestimmt

In [ ]:
BATCH_SIZE = 32
# bei vortrainiertem Model PIC_SIZE nicht ändern!!!
PIC_SIZE = 224 # Annahme viereckiges Bild zb. WEnn PIC_SIZE=500 dann sind die bilder 500x500 pixel;
EPOCH_ANZAHL = 50
LEARNING_RATE =

# Nicht zu dokumentieren
größe_trainingsdatensatz = 0.2 # prozentuale größe des trainingsdatensatz
größe_validierungsdatensatz = 0.2 # prozentuale größe des validierungsdatensatz

# Importe
Evtl. müssen hier bibliotheken installiert werden, wie z.B. pip install keras_tuner

In [ ]:
import tensorflow as tf
import time
from time import localtime
import os
import cv2
import imghdr
import matplotlib.pyplot as plt
import numpy as np
import random
from keras.callbacks import TensorBoard, CSVLogger, ModelCheckpoint, EarlyStopping
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Activation
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.model_selection import StratifiedShuffleSplit
from keras.callbacks import TensorBoard, CSVLogger, ModelCheckpoint, EarlyStopping

# Prüfung ob GPU genutzt wird --> siehe Ausgabe der Zelle

In [ ]:
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

seed_value = 0
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

GPU is available


# FKT um Trainingsergebnisse und eingestellte Parameter abzuspeichern abzuspeichern

In [ ]:
# neue path werden angelegt
def path_erzeuger(basepath, anzuhängender_foldername, zuspeichernder_file_name=None):
  path = os.path.join(basepath, anzuhängender_foldername)
  os.makedirs(path, exist_ok = True)
  return path

In [ ]:
def parameter_in_datei_speichern(ordner_pfad, datei_name, parameter_dict):
    datei_pfad = f'{ordner_pfad}/{datei_name}'

    # Öffnen der Datei im Schreibmodus ('w')
    with open(datei_pfad, 'w') as datei:
        # Schreiben der Parameter in die Datei
        for key, value in parameter_dict.items():
            datei.write(f'{key}: {value}\n')

In [ ]:
# zeiten
tag_monat = time.strftime("%d.%b", localtime())
stunde_minute = time.strftime("%Hh_%Mm", localtime())
stunde_minute_sekunde = time.strftime("%Hh_%Mm_%Ss", localtime())

### Path für Trainingsordner

In [ ]:
"""
# path für hptuning
callbacks_für_hptuning = path_erzeuger(basepath=base_path_für_logs_und_callbacks , anzuhängender_foldername = f"HP_Tuning_am_{tag_monat}")

# ordner im hptuning path
tensorboard_path_hptuning = path_erzeuger(callbacks_für_hptuning, f"TensorBoard_{stunde_minute_sekunde}")
hp_tuning_path = path_erzeuger(basepath=callbacks_für_hptuning, anzuhängender_foldername = f"hp_tuning_log_am_{stunde_minute_sekunde}")
"""

In [ ]:
#path für callbacks
callbacks_des_tages =  path_erzeuger(basepath=base_path_für_logs_und_callbacks , anzuhängender_foldername = f"Vortrainiertes_Modell_Callbacks_am_{tag_monat}") # für jeden tag wird ein ordern angelegt

# ordner im callbacks path
tensorboard_path_beste_modelle = path_erzeuger(callbacks_des_tages, f"TensorBoard_{stunde_minute_sekunde}")
checkpoint_path_für_bestes_model = path_erzeuger(callbacks_des_tages ,f"Checkpoint_{stunde_minute_sekunde}")

'\n#path für callbacks\ncallbacks_des_tages =  path_erzeuger(basepath=base_path_für_logs_und_callbacks , anzuhängender_foldername = f"Callbacks_am_{tag_monat}") # für jeden tag wird ein ordern angelegt\n\n# ordner im callbacks path\ntensorboard_path_beste_modelle = path_erzeuger(callbacks_des_tages, f"TensorBoard_{stunde_minute_sekunde}")\ncheckpoint_path_für_bestes_model = path_erzeuger(callbacks_des_tages ,f"Checkpoint_{stunde_minute_sekunde}")\n'

### Path für eingestellte Parameter

In [ ]:
festgelegte_parameter = {
    "learning_rate":LEARNING_RATE,
    "batch_size": BATCH_SIZE,
    "epochs": EPOCH_ANZAHL,
    "bildsize": PIC_SIZE,
    "testdatensatz_size":größe_trainingsdatensatz,
    "valdatensatz_size":größe_validierungsdatensatz,
}
"""
parameter_bei_hptuning = path_erzeuger(basepath=base_path_für_eingestellte_parameter , anzuhängender_foldername = f"HP_Tuning_am_{tag_monat}")
parameter_in_datei_speichern(ordner_pfad=parameter_bei_hptuning, datei_name=f"Parameter_um_{stunde_minute_sekunde}.txt", parameter_dict=festgelegte_parameter)

"""
parameter_bei_modeltraining = path_erzeuger(basepath=base_path_für_eingestellte_parameter , anzuhängender_foldername = f"Modeltraining_am_{tag_monat}")
parameter_in_datei_speichern(ordner_pfad=parameter_bei_modeltraining, datei_name=f"Parameter_um_{stunde_minute_sekunde}.txt", parameter_dict=festgelegte_parameter)

'\nparameter_bei_modeltraining = path_erzeuger(basepath=base_path_für_eingestellte_parameter , anzuhängender_foldername = f"Modeltraining_am_{tag_monat}")\nparameter_in_datei_speichern(ordner_pfad=parameter_bei_modeltraining, datei_name=f"Parameter_um_{stunde_minute_sekunde}.txt", parameter_dict=festgelegte_parameter)\n'

# Daten hochladen

In [ ]:
image_exts = ['jpeg', 'jpg', 'png']
# data_dir = 'data' # Für Windows-Rechner

for image_class in os.listdir(data_dir):
    class_path = os.path.join(data_dir, image_class)

    if os.path.isdir(class_path):
        for image in os.listdir(class_path):
            image_path = os.path.join(class_path, image)

            if os.path.isfile(image_path):
                _, file_ext = os.path.splitext(image_path) # check the file ending
                file_ext = file_ext[1:].lower()
                image_type = imghdr.what(image_path) # imghdr checks out if it is a image file

                if file_ext not in image_exts or not image_type:
                    print('File not in image extensions list or not a valid image: {}'.format(image_path))
                    os.remove(image_path)

# Preprocessing data

In [ ]:
data = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    image_size=(224, 224),
    label_mode="binary",
    shuffle=True,
    seed=0
)


Found 852 files belonging to 2 classes.


## Daten splitten in Train, Test, val

In [ ]:
# umwandlung in numpy array um die funktion StratifiedShuffleSplit von sklearn auf den datensatz anwenden zu können. Dadurch wird der das verhältnis der klassen bewahrt
features = []
labels = []
for image_batch, label_batch in data:
    features.extend(image_batch.numpy())
    labels.extend(label_batch.numpy())

features = np.array(features)
labels = np.array(labels)

# abspaltung testdaten
abspaltung_test = StratifiedShuffleSplit(n_splits=1, test_size=größe_trainingsdatensatz, random_state=0)
train_val_index, test_index = next(abspaltung_test.split(features, labels))
X_train_val, X_test = features[train_val_index], features[test_index]
y_train_val, y_test = labels[train_val_index], labels[test_index]

# aufteilung val und train
aufteilung_train_val = StratifiedShuffleSplit(n_splits=1, test_size=größe_validierungsdatensatz, random_state=0)
train_index, val_index = next(aufteilung_train_val.split(X_train_val, y_train_val))

# Aufteilung der Trainings- und Validierungsdaten
X_train, X_val = X_train_val[train_index], X_train_val[val_index]
y_train, y_val = y_train_val[train_index], y_train_val[val_index]

X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape


((544, 224, 224, 3),
 (544, 1),
 (137, 224, 224, 3),
 (137, 1),
 (171, 224, 224, 3),
 (171, 1))

## Daten verarbeiten

In [ ]:
X_train = preprocess_input(X_train)
X_val = preprocess_input(X_val)
X_test = preprocess_input(X_test)

In [ ]:
# Erstelle tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [ ]:
# shuffle, batch
train_data = train_dataset.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
test_data = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
val_data = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
"""
for element in val_data:
  print(element)
  break
"""

'\nfor element in val_data:\n  print(element)\n  break\n'

In [ ]:
"""
for image, label in train_dataset.take(5):
    plt.figure()
    img = image.numpy()
    img/=255
    plt.imshow(img)
    plt.title(f'Label: {label.numpy()}')
    plt.axis('off')
    plt.show()
"""

"\nfor image, label in train_dataset.take(5):\n    plt.figure()\n    img = image.numpy()\n    img/=255\n    plt.imshow(img)\n    plt.title(f'Label: {label.numpy()}')\n    plt.axis('off')\n    plt.show()\n"

# Model Erstellung

In [ ]:
vgg_model = tf.keras.applications.VGG16(weights="imagenet", include_top=True, input_shape=(224, 224, 3))

model = Sequential()
model.add(vgg_model)

# weitere layer müssen hinzugefügt werden --> Anzahl,Größe dieser Layer wird mit dem HP_tuning des Vortrainierten models bestimmt

model.add(Dense(units=, kernel_initializer="he_normal"))
model.add(Activation("relu"))
model.add(Dense(units=1))
model.add(Activation("softmax"))


553467096/553467096 [==============================] - 25s 0us/step


In [ ]:
model.compile(loss = "binary_crossentropy",
              optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), # learning_rate aus HP_TUNING
               metrics = ["accuracy"])

In [ ]:
# callbacks
best_model = ModelCheckpoint(checkpoint_path_für_bestes_model,monitor"val_auc", save_best_only=True, mode="max")
early_stop = EarlyStopping(monitor='val_acc',patience=10,mode='max')
tensorboard= TensorBoard(log_dir=tensorboard_path_beste_modelle)
alle_callbacks=[best_model,early_stop,tensorboard]

In [ ]:
model.fit(train_data, batch_size=BATCH_SIZE, validation_data=val_data, epochs=ANZAHL_EPOCHS, callbacks=alle_callbacks)